In [ ]:
import datetime
import os
import plotly.express as px
import streamlit as st
import mysql.connector
import pandas as pd
import base64
from PIL import Image,ImageFont,ImageDraw
from sshtunnel import SSHTunnelForwarder
import sshtunnel
from dotenv import load_dotenv
load_dotenv(".env")

### 複数テーブルの作成テスト

In [ ]:
def tenpo_name_list(sql):
        df = pd.read_csv('csv/tenpo_name_list.csv')
        return df

st.set_page_config(layout="wide")
st.title("ポートフォリオ用サイト(データ分析）")

yesterday = datetime.date.today() + datetime.timedelta(days=-3)
#yesterday = datetime.datetime(2022, 10, 10)
str_yesterday = yesterday.strftime("%Y-%m-%d")
st.write(yesterday)

kisyu_name = ''

print(os.getenv('SSH_USERNAME'))
with sshtunnel.SSHTunnelForwarder(
    (os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
    ssh_pkey="sercret/akasaka.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )
    ) as server:

    # SSH接続確認
    print(f"local bind port: {server.local_bind_port}")
    # データベース接続
    cnx = mysql.connector.connect(
        host="localhost", 
        port=server.local_bind_port, 
        user=os.getenv('WORDPRESS_DB_ID'), 
        password=os.getenv('DB_PASSWORD'), 
        database=os.getenv('WORDPRESS_DB_NAME'), 
        charset="utf8",
        use_pure=True
        )

    # 接続確認
    print(f"sql connection status: {cnx.is_connected()}")
    cursor = cnx.cursor()


    # cursor.execute(f"""
    # SELECT 
    # FROM {os.getenv('WORDPRESS_DB_TABLE')}
    # WHERE 日付 = '{yesterday.strftime('%Y-%m-%d')} 00:00:00' 
    # """)

    # #WHERE (店舗名 LIKE "%%") AND (日付 BETWEEN "{start_day}" AND "{last_day}") AND (機種名 LIKE "%{kisyu_name}%") 
    # cols = [col[0] for col in cursor.description]
    # all_tenpo__df = pd.DataFrame(cursor.fetchall(),columns = cols)
    #st.write(all_tenpo__df)

    st.sidebar.title('SLOT_DB_analytics')
    st.sidebar.title('複数店舗データ比較・検索用')
    st.title('※使い方がよくわからない人は左下の検索ボタンをそのまま押してください。')
    st.title('検索データが大きすぎると固まるため必ず都道府県か店舗名を一つ以上指定してください')
    sql = f"""
    SELECT 店舗名,都道府県
    FROM {os.getenv('WORDPRESS_DB_TABLE')}
    WHERE 日付 = '2023-04-10 00:00:00' 
    """

    #st.write(sql)
    df = tenpo_name_list(sql)
    #st.write(df)